# Spacecraft with reaction wheels and star tracker

Import modules and configure the notebook.

In [1]:
# Standard modules
import time
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym
from scipy import linalg

# My own script (an interface to the simulator)
import ae353_spacecraft

pybullet build time: Jan 28 2022 20:18:15


Create an instance of the `Simulator` class, which is an interface to the [pybullet](http://pybullet.org) simulation engine.

Here are two optional arguments you may want to use:

* `display` (`True` or `False`) is whether or not to show the simulation window;
* `seed` (an integer, default `None`) is a seed to initialize the random number generator used by the simulator.

**You must evaluate this cell only *once*.** If you want to start fresh with a new simulator, you must do `Kernel -> Restart` from the notebook menu first. If you evaluate this cell more than once without a call to `Kernel -> Restart` then you may get strange behavior that is hard to debug.

In [2]:
simulator = ae353_spacecraft.Simulator(
    display=False,
    seed=None,
)

Here is how to find the location of each star:

In [3]:
ra = []
dec = []
for (i, star) in enumerate(simulator.stars):
    ra.append(star['alpha'])
    dec.append(star['delta'])
    print(f'star {i} : alpha = {star["alpha"]:5.2f} : delta = {star["delta"]:5.2f}')

star 0 : alpha = -0.10 : delta = -0.15
star 1 : alpha =  0.00 : delta = -0.15
star 2 : alpha =  0.10 : delta = -0.15
star 3 : alpha =  0.00 : delta =  0.00
star 4 : alpha = -0.10 : delta =  0.15
star 5 : alpha =  0.00 : delta =  0.15
star 6 : alpha =  0.10 : delta =  0.15


# Define the Equations of Motion

Define Physical Parameters

In [4]:
# Mass and MOI of base
mb = 6.
Jxb = 10.
Jyb = 10.
Jzb = 16.

# Mass and MOI of each wheel
mw = 1.
Jxw = 0.075
Jyw = 0.075
Jzw = 0.125
lw = 1.1

Derive the Equation of Motion

In [5]:
# Define yaw, pitch, roll yaw angles
psi, theta, phi = sym.symbols('psi, theta, phi')

# Define angular velocities
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')

# Define torques
tau_1, tau_2, tau_3, tau_4 = sym.symbols('tau_1, tau_2, tau_3, tau_4')

# Compute resultant torques
lt = sym.nsimplify(lw) * sym.sqrt(2) / 2
T1 = - tau_1 * sym.Matrix([[lt], [0], [lt]])
T2 = - tau_2 * sym.Matrix([[-lt], [0], [lt]])
T3 = - tau_3 * sym.Matrix([[0], [lt], [lt]])
T4 = - tau_4 * sym.Matrix([[0], [-lt], [lt]])
T = T1 + T2 + T3 + T4

# Define MOI of spacecraft and wheels together
Jx = sym.nsimplify(Jxb + 4 * mw * lw**2)
Jy = sym.nsimplify(Jyb + 4 * mw * lw**2)
Jz = sym.nsimplify(Jzb + 4 * mw * lw**2)

# Define rotation matrices
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0], [sym.sin(psi), sym.cos(psi), 0], [0, 0, 1]])
Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)], [0, 1, 0], [-sym.sin(theta), 0, sym.cos(theta)]])
Rx = sym.Matrix([[1, 0, 0], [0, sym.cos(phi), -sym.sin(phi)], [0, sym.sin(phi), sym.cos(phi)]])

# Define the transformation from angular velocity to angular rates
ex = sym.Matrix([[1], [0], [0]])
ey = sym.Matrix([[0], [1], [0]])
ez = sym.Matrix([[0], [0], [1]])
M = sym.simplify(sym.Matrix.hstack((Ry @ Rx).T @ ez, Rx.T @ ey, ex).inv(), full=True)

# Define euler's equations
euler = sym.Matrix([[(1 / Jx) * (T[0] + (Jy - Jz) * w_y * w_z)],
                   [(1 / Jy) * (T[1] + (Jz - Jx) * w_z * w_x)],
                   [(1 / Jz) * (T[2] + (Jx - Jy) * w_x * w_y)]])

# Define equations of motion
f = sym.simplify(sym.Matrix.vstack(M * sym.Matrix([[w_x], [w_y], [w_z]]), euler), full=True)

The equations of motion have this form:

$$\begin{bmatrix} \dot{\psi} \\ \dot{\theta} \\ \dot{\phi} \\ \dot{w_x} \\ \dot{w_y} \\ \dot{w_z} \end{bmatrix} = f\left(\psi, \theta, \phi, w_x, w_y, w_z, \tau_1, \tau_2, \tau_3, \tau_4\right)$$

Here is the function $f$:

In [6]:
f

Matrix([
[                        (w_y*sin(phi) + w_z*cos(phi))/cos(theta)],
[                                     w_y*cos(phi) - w_z*sin(phi)],
[         w_x + w_y*sin(phi)*tan(theta) + w_z*cos(phi)*tan(theta)],
[-55*sqrt(2)*tau_1/1484 + 55*sqrt(2)*tau_2/1484 - 150*w_y*w_z/371],
[-55*sqrt(2)*tau_3/1484 + 55*sqrt(2)*tau_4/1484 + 150*w_x*w_z/371],
[                -55*sqrt(2)*(tau_1 + tau_2 + tau_3 + tau_4)/2084]])

## Sensor Model

In [7]:
alpha, delta = sym.symbols('alpha, delta')

# Define the Physical Parameters
# Scope radius
r = 0.8 / 2.1

Derive the Sensor Model

In [8]:
def g_matrix(alpha,delta):
    # Position of star in space frame
    p_star_in_space = sym.Matrix([[sym.cos(alpha) * sym.cos(delta)],
                                  [sym.sin(alpha) * sym.cos(delta)],
                                  [sym.sin(delta)]])

    # Orientation of body frame in space frame
    R_body_in_space = Rz * Ry * Rx

    # Position of star in body frame (assuming origin of body and space frames are the same)
    p_star_in_body = R_body_in_space.T * p_star_in_space

    # Position of star in image frame
    p_star_in_image = (1 / sym.nsimplify(r)) * sym.Matrix([[p_star_in_body[1] / p_star_in_body[0]],
                                                           [p_star_in_body[2] / p_star_in_body[0]]])

    # Sensor model for each star
    g = sym.simplify(p_star_in_image, full=True)
    return g

In [9]:
# Position of star in space frame
p_star_in_space = sym.Matrix([[sym.cos(alpha) * sym.cos(delta)],
                              [sym.sin(alpha) * sym.cos(delta)],
                                [sym.sin(delta)]])

    # Orientation of body frame in space frame
R_body_in_space = Rz * Ry * Rx

    # Position of star in body frame (assuming origin of body and space frames are the same)
p_star_in_body = R_body_in_space.T * p_star_in_space

    # Position of star in image frame
p_star_in_image = (1 / sym.nsimplify(r)) * sym.Matrix([[p_star_in_body[1] / p_star_in_body[0]],
                                                           [p_star_in_body[2] / p_star_in_body[0]]])

    # Sensor model for each star
g = sym.simplify(p_star_in_image, full=True)
g

Matrix([
[21*(sin(delta)*sin(phi)*cos(theta) + sin(phi)*sin(theta)*cos(delta)*cos(alpha - psi) + sin(alpha - psi)*cos(delta)*cos(phi))/(8*(-sin(delta)*sin(theta) + cos(delta)*cos(theta)*cos(alpha - psi)))],
[21*(sin(delta)*cos(phi)*cos(theta) - sin(phi)*sin(alpha - psi)*cos(delta) + sin(theta)*cos(delta)*cos(phi)*cos(alpha - psi))/(8*(-sin(delta)*sin(theta) + cos(delta)*cos(theta)*cos(alpha - psi)))]])

In [10]:
print(ra)
print(dec)

[-0.1, 0.0, 0.1, 0.0, -0.1, 0.0, 0.1]
[-0.15, -0.15, -0.15, 0.0, 0.15, 0.15, 0.15]


In [11]:
gs = []
for i in range(len(ra)):
    matrix_g = g_matrix(ra[i],dec[i])
    gs.append(matrix_g)
g = sym.Matrix(np.array([
    [gs[0][0]],
    [gs[0][1]],
    [gs[1][0]],
    [gs[1][1]],
    [gs[2][0]],
    [gs[2][1]],
    [gs[3][0]],
    [gs[3][1]],
    [gs[4][0]],
    [gs[4][1]],
    [gs[5][0]],
    [gs[5][1]],
    [gs[6][0]],
    [gs[6][1]],
]))
    

The sensor model has this form for each star:

$$\zeta = g(\psi, \theta, \phi, \alpha, \delta)$$

Here is the function $g$ accounting for all the stars:

In [12]:
from sympy.physics import mechanics
mechanics.init_vprinting()
symg = sym.Matrix(g)
# print(f'{mechanics.mlatex(symg)}')

[\begin{matrix}\frac{- 2.07296029483037 \sin{\left(\phi \right)} \sin{\left(\psi \right)} \sin{\left(\theta \right)} + 20.6604581621089 \sin{\left(\phi \right)} \sin{\left(\theta \right)} \cos{\left(\psi \right)} - 3.13820078194558 \sin{\left(\phi \right)} \cos{\left(\theta \right)} - 20.6604581621089 \sin{\left(\psi \right)} \cos{\left(\phi \right)} - 2.07296029483037 \cos{\left(\phi \right)} \cos{\left(\psi \right)}}{- 0.789699159935378 \sin{\left(\psi \right)} \cos{\left(\theta \right)} + 1.19550505978879 \sin{\left(\theta \right)} + 7.87065072842244 \cos{\left(\psi \right)} \cos{\left(\theta \right)}}\\\frac{20.6604581621089 \sin{\left(\phi \right)} \sin{\left(\psi \right)} + 2.07296029483037 \sin{\left(\phi \right)} \cos{\left(\psi \right)} - 2.07296029483037 \sin{\left(\psi \right)} \sin{\left(\theta \right)} \cos{\left(\phi \right)} + 20.6604581621089 \sin{\left(\theta \right)} \cos{\left(\phi \right)} \cos{\left(\psi \right)} - 3.13820078194558 \cos{\left(\phi \right)} \cos{\left(\theta \right)}}{- 0.789699159935378 \sin{\left(\psi \right)} \cos{\left(\theta \right)} + 1.19550505978879 \sin{\left(\theta \right)} + 7.87065072842244 \cos{\left(\psi \right)} \cos{\left(\theta \right)}}\\\frac{20.7641926366569 \sin{\left(\phi \right)} \sin{\left(\theta \right)} \cos{\left(\psi \right)} - 3.13820078194558 \sin{\left(\phi \right)} \cos{\left(\theta \right)} - 20.7641926366569 \sin{\left(\psi \right)} \cos{\left(\phi \right)}}{1.19550505978879 \sin{\left(\theta \right)} + 7.91016862348834 \cos{\left(\psi \right)} \cos{\left(\theta \right)}}\\\frac{20.7641926366569 \sin{\left(\phi \right)} \sin{\left(\psi \right)} + 20.7641926366569 \sin{\left(\theta \right)} \cos{\left(\phi \right)} \cos{\left(\psi \right)} - 3.13820078194558 \cos{\left(\phi \right)} \cos{\left(\theta \right)}}{1.19550505978879 \sin{\left(\theta \right)} + 7.91016862348834 \cos{\left(\psi \right)} \cos{\left(\theta \right)}}\\\frac{2.07296029483037 \sin{\left(\phi \right)} \sin{\left(\psi \right)} \sin{\left(\theta \right)} + 20.6604581621089 \sin{\left(\phi \right)} \sin{\left(\theta \right)} \cos{\left(\psi \right)} - 3.13820078194558 \sin{\left(\phi \right)} \cos{\left(\theta \right)} - 20.6604581621089 \sin{\left(\psi \right)} \cos{\left(\phi \right)} + 2.07296029483037 \cos{\left(\phi \right)} \cos{\left(\psi \right)}}{0.789699159935378 \sin{\left(\psi \right)} \cos{\left(\theta \right)} + 1.19550505978879 \sin{\left(\theta \right)} + 7.87065072842244 \cos{\left(\psi \right)} \cos{\left(\theta \right)}}\\\frac{20.6604581621089 \sin{\left(\phi \right)} \sin{\left(\psi \right)} - 2.07296029483037 \sin{\left(\phi \right)} \cos{\left(\psi \right)} + 2.07296029483037 \sin{\left(\psi \right)} \sin{\left(\theta \right)} \cos{\left(\phi \right)} + 20.6604581621089 \sin{\left(\theta \right)} \cos{\left(\phi \right)} \cos{\left(\psi \right)} - 3.13820078194558 \cos{\left(\phi \right)} \cos{\left(\theta \right)}}{0.789699159935378 \sin{\left(\psi \right)} \cos{\left(\theta \right)} + 1.19550505978879 \sin{\left(\theta \right)} + 7.87065072842244 \cos{\left(\psi \right)} \cos{\left(\theta \right)}}\\\frac{21 \sin{\left(\phi \right)} \tan{\left(\theta \right)}}{8} - \frac{21 \cos{\left(\phi \right)} \tan{\left(\psi \right)}}{8 \cos{\left(\theta \right)}}\\\frac{21 \sin{\left(\phi \right)} \tan{\left(\psi \right)}}{8 \cos{\left(\theta \right)}} + \frac{21 \cos{\left(\phi \right)} \tan{\left(\theta \right)}}{8}\\\frac{2.07296029483037 \sin{\left(\phi \right)} \sin{\left(\psi \right)} \sin{\left(\theta \right)} - 20.6604581621089 \sin{\left(\phi \right)} \sin{\left(\theta \right)} \cos{\left(\psi \right)} - 3.13820078194558 \sin{\left(\phi \right)} \cos{\left(\theta \right)} + 20.6604581621089 \sin{\left(\psi \right)} \cos{\left(\phi \right)} + 2.07296029483037 \cos{\left(\phi \right)} \cos{\left(\psi \right)}}{0.789699159935378 \sin{\left(\psi \right)} \cos{\left(\theta \right)} + 1.19550505978879 \sin{\left(\theta \right)} - 7.87065072842244 \cos{\left(\psi \right)} \cos{\left(\theta \right)}}\\\frac{- 20.6604581621089 \sin{\left(\phi \right)} \sin{\left(\psi \right)} - 2.07296029483037 \sin{\left(\phi \right)} \cos{\left(\psi \right)} + 2.07296029483037 \sin{\left(\psi \right)} \sin{\left(\theta \right)} \cos{\left(\phi \right)} - 20.6604581621089 \sin{\left(\theta \right)} \cos{\left(\phi \right)} \cos{\left(\psi \right)} - 3.13820078194558 \cos{\left(\phi \right)} \cos{\left(\theta \right)}}{0.789699159935378 \sin{\left(\psi \right)} \cos{\left(\theta \right)} + 1.19550505978879 \sin{\left(\theta \right)} - 7.87065072842244 \cos{\left(\psi \right)} \cos{\left(\theta \right)}}\\\frac{- 20.7641926366569 \sin{\left(\phi \right)} \sin{\left(\theta \right)} \cos{\left(\psi \right)} - 3.13820078194558 \sin{\left(\phi \right)} \cos{\left(\theta \right)} + 20.7641926366569 \sin{\left(\psi \right)} \cos{\left(\phi \right)}}{1.19550505978879 \sin{\left(\theta \right)} - 7.91016862348834 \cos{\left(\psi \right)} \cos{\left(\theta \right)}}\\\frac{20.7641926366569 \sin{\left(\phi \right)} \sin{\left(\psi \right)} + 20.7641926366569 \sin{\left(\theta \right)} \cos{\left(\phi \right)} \cos{\left(\psi \right)} + 3.13820078194558 \cos{\left(\phi \right)} \cos{\left(\theta \right)}}{- 1.19550505978879 \sin{\left(\theta \right)} + 7.91016862348834 \cos{\left(\psi \right)} \cos{\left(\theta \right)}}\\\frac{2.07296029483037 \sin{\left(\phi \right)} \sin{\left(\psi \right)} \sin{\left(\theta \right)} + 20.6604581621089 \sin{\left(\phi \right)} \sin{\left(\theta \right)} \cos{\left(\psi \right)} + 3.13820078194558 \sin{\left(\phi \right)} \cos{\left(\theta \right)} - 20.6604581621089 \sin{\left(\psi \right)} \cos{\left(\phi \right)} + 2.07296029483037 \cos{\left(\phi \right)} \cos{\left(\psi \right)}}{0.789699159935378 \sin{\left(\psi \right)} \cos{\left(\theta \right)} - 1.19550505978879 \sin{\left(\theta \right)} + 7.87065072842244 \cos{\left(\psi \right)} \cos{\left(\theta \right)}}\\\frac{20.6604581621089 \sin{\left(\phi \right)} \sin{\left(\psi \right)} - 2.07296029483037 \sin{\left(\phi \right)} \cos{\left(\psi \right)} + 2.07296029483037 \sin{\left(\psi \right)} \sin{\left(\theta \right)} \cos{\left(\phi \right)} + 20.6604581621089 \sin{\left(\theta \right)} \cos{\left(\phi \right)} \cos{\left(\psi \right)} + 3.13820078194558 \cos{\left(\phi \right)} \cos{\left(\theta \right)}}{0.789699159935378 \sin{\left(\psi \right)} \cos{\left(\theta \right)} - 1.19550505978879 \sin{\left(\theta \right)} + 7.87065072842244 \cos{\left(\psi \right)} \cos{\left(\theta \right)}}\end{matrix}]

In [13]:
g

⎡-2.07296029483037⋅sin(φ)⋅sin(ψ)⋅sin(θ) + 20.6604581621089⋅sin(φ)⋅sin(θ)⋅cos(ψ
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                           -0.789699159935378⋅sin(ψ)⋅cos(θ) +
⎢                                                                             
⎢20.6604581621089⋅sin(φ)⋅sin(ψ) + 2.07296029483037⋅sin(φ)⋅cos(ψ) - 2.072960294
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                          -0.789699159935378⋅sin(ψ)⋅cos(θ) + 
⎢                                                                             
⎢                                     20.7641926366569⋅sin(φ)⋅sin(θ)⋅cos(ψ) - 
⎢                                     ────────────────────────────────────────
⎢                                                             1.19550505978879
⎢                                                                             
⎢                                     20.76419263665

## Define Equilibrium:

In [14]:
# define equilibrium
psi_e = 0
theta_e = 0
phi_e = 0

w_x_e = 0
w_y_e = 0
w_z_e = 0

tau_1_e = 0
tau_2_e = 0
tau_3_e = 0
tau_4_e = 0





x_eq = np.array([
    psi_e,
    theta_e,
    phi_e,
    w_x_e,
    w_y_e,
    w_z_e     
        ])
u_eq = np.array([
    tau_1_e,
    tau_2_e,
    tau_3_e,
    tau_4_e
])

# check equilibrium
f_num = sym.lambdify([psi,theta,phi,w_x,w_y,w_z,tau_1,tau_2,tau_3,tau_4],f)
print('f_num =',f_num(psi_e,theta_e,phi_e,w_x_e,w_y_e,w_z_e,tau_1_e,tau_2_e,tau_3_e,tau_4_e)) # output should be zero

g_num = sym.lambdify([psi,theta,phi,w_x,w_y,w_z,tau_1,tau_2,tau_3,tau_4],g)
print('g_num =',g_num(psi_e,theta_e,phi_e,w_x_e,w_y_e,w_z_e,tau_1_e,tau_2_e,tau_3_e,tau_4_e)) # output shouldnt be zero 

f_num = [[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-0.]]
g_num = [[-0.26337851]
 [-0.3987219 ]
 [ 0.        ]
 [-0.39672995]
 [ 0.26337851]
 [-0.3987219 ]
 [ 0.        ]
 [ 0.        ]
 [-0.26337851]
 [ 0.3987219 ]
 [-0.        ]
 [ 0.39672995]
 [ 0.26337851]
 [ 0.3987219 ]]


Find A and B an C Matrices

In [33]:
# Assemble state and input matrices
A_num = sym.lambdify((psi,theta,phi,w_x,w_y,w_z,tau_1,tau_2,tau_3,tau_4), f.jacobian([psi,theta,phi,w_x,w_y,w_z]))
B_num = sym.lambdify((psi,theta,phi,w_x,w_y,w_z,tau_1,tau_2,tau_3,tau_4), f.jacobian([tau_1,tau_2,tau_3,tau_4]))
C_num = sym.lambdify((psi,theta,phi,w_x,w_y,w_z,tau_1,tau_2,tau_3,tau_4), g.jacobian([psi,theta,phi,w_x,w_y,w_z])) 

A = A_num(psi_e,theta_e,phi_e,w_x_e,w_y_e,w_z_e,tau_1_e,tau_2_e,tau_3_e,tau_4_e)
B = B_num(psi_e,theta_e,phi_e,w_x_e,w_y_e,w_z_e,tau_1_e,tau_2_e,tau_3_e,tau_4_e)
C = C_num(psi_e,theta_e,phi_e,w_x_e,w_y_e,w_z_e,tau_1_e,tau_2_e,tau_3_e,tau_4_e)

print('A = ',A)
print('B = ',B)
print('C = ',C)

A =  [[ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1. -0.]
 [ 0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0. -0. -0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
B =  [[ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [-0.05241358  0.05241358  0.          0.        ]
 [ 0.          0.         -0.05241358  0.05241358]
 [-0.03732329 -0.03732329 -0.03732329 -0.03732329]]
C =  [[-2.651426    0.04000563 -0.3987219   0.          0.          0.        ]
 [-0.04000563  2.68556349  0.26337851  0.          0.          0.        ]
 [-2.625       0.         -0.39672995  0.          0.          0.        ]
 [ 0.          2.68495987  0.          0.          0.          0.        ]
 [-2.651426   -0.04000563 -0.3987219   0.          0.          0.        ]
 [ 0.04000563  2.68556349 -0.26337851  0.          0.          0.        ]
 [-2.625       0.          0.          0.          0.          0.

## Controllability

In [34]:
# Construct Controllability Matrix W
W = B
n = A.shape[0]

# Create W one column at a time by iterating over i from 1 to n-1
for i in range(1, n):
    col = np.linalg.matrix_power(A, i) @ B
    W = np.block([W, col])
W

# determines if a system is controllable or not    
def is_controllable(A,B):
    W = B
    n = A.shape[0]

    for i in range(1, n):
        col = np.linalg.matrix_power(A, i) @ B
        W = np.block([W, col])
    r = np.linalg.matrix_rank(W)
    if r==W.shape[0]:
        return print('Controllable!')
    else:
        return print('Not Controllable!')
is_controllable(A,B)

Controllable!


## Observability

In [35]:
def is_observable(A,C):
    A = A.T
    C = C.T
    O = C
    n = A.shape[0]

    for i in range(1, n):
        col = np.linalg.matrix_power(A, i) @ C
        O = np.block([O, col])
    r = np.linalg.matrix_rank(O)
    if r==O.shape[0]:
        return print('Observable!')
    else:
        return print('Not Observable!')
is_observable(A,C)

Observable!


# LQR

In [36]:
# Construct K
from sympy.physics import mechanics
mechanics.init_vprinting()
symg = sym.Matrix(g)
#print(f'{mechanics.mlatex(symg)}')

# Define the weights
QK = np.eye(len(A[0]))
QK[0,0] = 500     # psi weight
QK[1,1] = 500     # theta weight
QK[2,2] = 500     # phi weight
QK[3,3] = 100     # w_x weight
QK[4,4] = 100     # w_y weight
QK[5,5] = 100    # w_z weight
# symQK = sym.Matrix(QK)
# print(f'Q_K = {mechanics.mlatex(symQK)}')


RK = np.eye(4) * 100
# symRK = sym.Matrix(RK)
# print(f'R_K = {mechanics.mlatex(symRK)}')


# Find the optimal cost matrix and gain matrix (both should be 2D NumPy arrays)
PK = linalg.solve_continuous_are(A, B, QK, RK)
# symPK = sym.Matrix(PK)
# print(f'P_K = {mechanics.mlatex(symPK)}')

K = linalg.inv(RK) @  B.T @ PK
# symK = sym.Matrix(K)
# print(f'K = {mechanics.mlatex(symK)}')

F = A-B@K
s = linalg.eigvals(F)
if (s.real < 0).all():
    print("Asymptotically Stable")

Asymptotically Stable


$Q_K = \begin{matrix}500.0 & 0 & 0 & 0 & 0 & 0\\0 & 500.0 & 0 & 0 & 0 & 0\\0 & 0 & 500.0 & 0 & 0 & 0\\0 & 0 & 0 & 100.0 & 0 & 0\\0 & 0 & 0 & 0 & 100.0 & 0\\0 & 0 & 0 & 0 & 0 & 100.0\end{matrix}$

$R_K = \begin{matrix}100.0 & 0 & 0 & 0\\0 & 100.0 & 0 & 0\\0 & 0 & 100.0 & 0\\0 & 0 & 0 & 100.0\end{matrix}$

$P_K = \begin{matrix}1745.14738161031 & 1.01346916445665 \cdot 10^{-12} & 5.05677433506087 \cdot 10^{-13} & 1.3554332599878 \cdot 10^{-12} & 3.84784644385262 \cdot 10^{-12} & 2995.53938354132\\1.01346916445665 \cdot 10^{-12} & 1751.18787700071 & -2.8066438062524 \cdot 10^{-13} & -2.03082184098241 \cdot 10^{-11} & 3016.65898055425 & 1.6804683533958 \cdot 10^{-12}\\5.05677433506087 \cdot 10^{-13} & -2.8066438062524 \cdot 10^{-13} & 1751.18787700071 & 3016.65898055425 & 1.06723518911167 \cdot 10^{-11} & 4.03575555918232 \cdot 10^{-12}\\1.3554332599878 \cdot 10^{-12} & -2.03082184098241 \cdot 10^{-11} & 3016.65898055425 & 10565.4732715839 & -3.58457444892036 \cdot 10^{-11} & 1.8142534465407 \cdot 10^{-11}\\3.84784644385262 \cdot 10^{-12} & 3016.65898055425 & 1.06723518911167 \cdot 10^{-11} & -3.58457444892036 \cdot 10^{-11} & 10565.4732715838 & 7.02341585371255 \cdot 10^{-12}\\2995.53938354132 & 1.6804683533958 \cdot 10^{-12} & 4.03575555918232 \cdot 10^{-12} & 1.8142534465407 \cdot 10^{-11} & 7.02341585371255 \cdot 10^{-12} & 10455.3154233954\end{matrix}$

$K = \begin{matrix}-1.1180339887499 & 1.00170572196127 \cdot 10^{-14} & -1.58113883008419 & -5.53774230219703 & 1.6166666134929 \cdot 10^{-14} & -3.90226817603644\\-1.1180339887499 & -1.12714695277519 \cdot 10^{-14} & 1.58113883008418 & 5.53774230219701 & -2.14094065139872 \cdot 10^{-14} & -3.90226817603642\\-1.1180339887499 & -1.58113883008419 & -7.10003814444092 \cdot 10^{-15} & 1.20166447394102 \cdot 10^{-14} & -5.53774230219702 & -3.90226817603643\\-1.11803398874989 & 1.58113883008419 & 4.08748427167696 \cdot 10^{-15} & -2.55594279095061 \cdot 10^{-14} & 5.53774230219701 & -3.90226817603642\end{matrix}$

In [44]:
# Construct L

# Define the weights
Q_obs = np.eye(14)
# symQL = sym.Matrix(QL)
# print(f'Q_L = {mechanics.mlatex(symQL)}')


R_obs = np.eye(6)
# symRL = sym.Matrix(RL)
# print(f'R_L = {mechanics.mlatex(symRL)}')

# Find the optimal cost matrix and gain matrix (both should be 2D NumPy arrays)
P_obs = linalg.solve_continuous_are(A.T, C.T, linalg.inv(R_obs), linalg.inv(Q_obs))


# symPL = sym.Matrix(PK)
# print(f'P_K = {mechanics.mlatex(symPL)}')

L = P_obs@C.T@Q_obs
# symL = sym.Matrix(L)
# print(f'L = {mechanics.mlatex(symL)}')

F = A.T-C.T@L.T
s = linalg.eigvals(F)
if (s.real < 0).all():
    print("Asymptotically Stable")


Asymptotically Stable


$Q_L = \begin{matrix}1000.0 & 0 & 0 & 0 & 0 & 0\\0 & 500.0 & 0 & 0 & 0 & 0\\0 & 0 & 1.0 & 0 & 0 & 0\\0 & 0 & 0 & 1.0 & 0 & 0\\0 & 0 & 0 & 0 & 1.0 & 0\\0 & 0 & 0 & 0 & 0 & 1.0\end{matrix}$

$R_L = \begin{array}{cccccccccccccc}2.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 2.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 2.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 2.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 2.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 2.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 2.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 2.0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 2.0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 2.0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 2.0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 2.0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 2.0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 2.0\end{array}$

$P_K = \begin{matrix}1745.14738161031 & 1.01346916445665 \cdot 10^{-12} & 5.05677433506087 \cdot 10^{-13} & 1.3554332599878 \cdot 10^{-12} & 3.84784644385262 \cdot 10^{-12} & 2995.53938354132\\1.01346916445665 \cdot 10^{-12} & 1751.18787700071 & -2.8066438062524 \cdot 10^{-13} & -2.03082184098241 \cdot 10^{-11} & 3016.65898055425 & 1.6804683533958 \cdot 10^{-12}\\5.05677433506087 \cdot 10^{-13} & -2.8066438062524 \cdot 10^{-13} & 1751.18787700071 & 3016.65898055425 & 1.06723518911167 \cdot 10^{-11} & 4.03575555918232 \cdot 10^{-12}\\1.3554332599878 \cdot 10^{-12} & -2.03082184098241 \cdot 10^{-11} & 3016.65898055425 & 10565.4732715839 & -3.58457444892036 \cdot 10^{-11} & 1.8142534465407 \cdot 10^{-11}\\3.84784644385262 \cdot 10^{-12} & 3016.65898055425 & 1.06723518911167 \cdot 10^{-11} & -3.58457444892036 \cdot 10^{-11} & 10565.4732715838 & 7.02341585371255 \cdot 10^{-12}\\2995.53938354132 & 1.6804683533958 \cdot 10^{-12} & 4.03575555918232 \cdot 10^{-12} & 1.8142534465407 \cdot 10^{-11} & 7.02341585371255 \cdot 10^{-12} & 10455.3154233954\end{matrix}$

$L = \begin{matrix}-8.48885486431936 & 0.0893445757647138 & -0.479482537208737 & -0.254404200195196 & 0.00399401622292677 & -0.268386832325127\\-0.128082772498207 & 5.99766898179153 & 0.316725515912345 & 0.16804845912879 & 0.26811686113007 & -0.0040495131717544\\-8.40424890049191 & 5.50401192672774 \cdot 10^{-17} & -0.477087121700769 & -0.253133238858444 & -2.09222918352056 \cdot 10^{-15} & -0.265711898309833\\-7.80998686473427 \cdot 10^{-17} & 5.99632091999115 & 1.44258441441759 \cdot 10^{-16} & -5.64092701103411 \cdot 10^{-17} & 0.268056598034586 & 5.11577944259968 \cdot 10^{-15}\\-8.48885486431936 & -0.0893445757647137 & -0.479482537208737 & -0.254404200195196 & -0.00399401622293098 & -0.268386832325127\\0.128082772498207 & 5.99766898179153 & -0.316725515912345 & -0.16804845912879 & 0.268116861130068 & 0.00404951317176464\\-8.40424890049191 & 7.6355761481033 \cdot 10^{-17} & 7.46719454473022 \cdot 10^{-17} & 6.64713044847239 \cdot 10^{-17} & -3.86248488153962 \cdot 10^{-16} & -0.265711898309833\\-7.6355761481033 \cdot 10^{-17} & 5.86241254768296 & 1.4103689720773 \cdot 10^{-16} & -5.51495520858462 \cdot 10^{-17} & 0.26207042364395 & 5.0015351072169 \cdot 10^{-15}\\-8.48885486431936 & -0.0893445757647136 & 0.479482537208737 & 0.254404200195196 & -0.00399401622292755 & -0.268386832325128\\0.128082772498207 & 5.99766898179153 & 0.316725515912345 & 0.16804845912879 & 0.26811686113007 & 0.00404951317176408\\-8.40424890049191 & 9.76714036947887 \cdot 10^{-17} & 0.477087121700769 & 0.253133238858445 & 1.31973220721264 \cdot 10^{-15} & -0.265711898309833\\-7.80998686473427 \cdot 10^{-17} & 5.99632091999115 & 1.44258441441759 \cdot 10^{-16} & -5.64092701103411 \cdot 10^{-17} & 0.268056598034586 & 5.11577944259968 \cdot 10^{-15}\\-8.48885486431936 & 0.0893445757647139 & 0.479482537208737 & 0.254404200195196 & 0.0039940162229302 & -0.268386832325128\\-0.128082772498207 & 5.99766898179153 & -0.316725515912345 & -0.16804845912879 & 0.268116861130068 & -0.00404951317175385\end{matrix}$

Define a controller that maps sensor measurements to actuator commands. By default, it applies zero torque to each reaction wheel. Modify the `run` function to make the controller behave differently.

In [ ]:
class Controller:
    def __init__(self):
        pass
    
    def reset(self):
        pass
    
    def run(self, t, star_meas):
        """
        star_meas is a 1d array of length 14 - the image coordinates
        y_i and z_i of the i'th star (for i = 0, ..., 6) are at index
        2*i and 2*i+1 of this array, respectively
        """
        
        # FIXME: replace the following lines
        front_torque = 0.
        back_torque = 0.
        left_torque = 0.
        right_torque = 0.
        
        
        return front_torque, back_torque, left_torque, right_torque

Create an instance of the controller.

In [ ]:
controller = Controller()

You could reset the simulator with given initial conditions, optionally specifying scope noise (default is `0.1`), i.e., the standard deviation of the $y$ and $z$ components of each star tracker measurement.

In [ ]:
simulator.reset(
    orientation={
        'yaw': 0.,
        'pitch': 0.,
        'roll': 0.,
    },
    angular_velocity={
        'x': 0.,
        'y': 0.,
        'z': 0.,
    },
    scope_noise=0.1,
)

Or, you could reset the simulator with initial conditions sampled at random.

In [ ]:
simulator.reset(
    orientation=None,
    angular_velocity=None,
    scope_noise=0.1,
)

Reset the controller.

In [ ]:
controller.reset()

Run the simulator. It will quit when one of the following conditions is satisfied:

* Any star goes out of view of the scope.
* Any reaction wheel has angular velocity exceeding +/- 50 rad/s.
* The maximum time is reached.

In [ ]:
data = simulator.run(
    controller,           # <-- required (an instance of your Controller class)
    max_time=5.0,         # <-- optional (how long you want to run the simulation in seconds)
    data_filename=None,   # <-- optional (name of file to which you want data saved, e.g., 'my_data.json')
    video_filename=None,  # <-- optional (name of file to which you want video saved, e.g., 'my_video.mov')
    print_debug=False,    # <-- optional (whether to print debug text - this is recommended when saving video)
)

Get, show, and save a snapshot of the simulation after the last time step.

In [ ]:
# Get snapshot as height x width x 4 numpy array of RGBA values
rgba = simulator.snapshot()

# Display snapshot
plt.figure(figsize=(8, 8))
plt.imshow(rgba)

# Save snapshot
plt.imsave('my_snapshot.png', rgba)

Plot the results.

In [ ]:
# Create a figure with three subplots, all of which share the same x-axis
fig, (ax_ori, ax_vel, ax_rwvel, ax_rwtau) = plt.subplots(4, 1, figsize=(10, 10), sharex=True)

# Plot yaw, pitch, roll angles
ax_ori.plot(data['t'], data['yaw'], label='yaw (rad)', linewidth=4)
ax_ori.plot(data['t'], data['pitch'], label='pitch (rad)', linewidth=4)
ax_ori.plot(data['t'], data['roll'], label='roll (rad)', linewidth=4)
ax_ori.grid()
ax_ori.legend(fontsize=16, ncol=3, loc='upper right')
ax_ori.tick_params(labelsize=14)

# Plot x, y, z components of angular velocity
ax_vel.plot(data['t'], data['w_x'], label=r'$w_x$ (rad/s)', linewidth=4)
ax_vel.plot(data['t'], data['w_y'], label=r'$w_y$ (rad/s)', linewidth=4)
ax_vel.plot(data['t'], data['w_z'], label=r'$w_z$ (rad/s)', linewidth=4)
ax_vel.grid()
ax_vel.legend(fontsize=16, ncol=3, loc='upper right')
ax_vel.tick_params(labelsize=14)

# Plot angular velocity of each reaction wheel
ax_rwvel.plot(data['t'], data['front_velocity'], label=r'$v_1$ (rad/s)', linewidth=4)
ax_rwvel.plot(data['t'], data['back_velocity'], label=r'$v_2$ (rad/s)', linewidth=4)
ax_rwvel.plot(data['t'], data['left_velocity'], label=r'$v_3$ (rad/s)', linewidth=4)
ax_rwvel.plot(data['t'], data['right_velocity'], label=r'$v_4$ (rad/s)', linewidth=4)
ax_rwvel.plot(
    data['t'], np.ones_like(data['t']) * simulator.v_max,
    ':', linewidth=4, color='C4', zorder=0,
)
ax_rwvel.plot(
    data['t'], -np.ones_like(data['t']) * simulator.v_max,
    ':', linewidth=4, color='C4', zorder=0,
)
ax_rwvel.grid()
ax_rwvel.legend(fontsize=16, ncol=4, loc='upper right')
ax_rwvel.tick_params(labelsize=14)
ax_rwvel.set_ylim(-1.2 * simulator.v_max, 1.2 * simulator.v_max)

# Plot torque applied to each reaction wheel
ax_rwtau.plot(data['t'], data['front_torque'], label=r'$\tau_1$ (N-m)', linewidth=4)
ax_rwtau.plot(data['t'], data['back_torque'], label=r'$\tau_2$ (N-m)', linewidth=4)
ax_rwtau.plot(data['t'], data['left_torque'], label=r'$\tau_3$ (N-m)', linewidth=4)
ax_rwtau.plot(data['t'], data['right_torque'], label=r'$\tau_4$ (N-m)', linewidth=4)
ax_rwtau.plot(
    data['t'], np.ones_like(data['t']) * simulator.tau_max,
    ':', linewidth=4, color='C4', zorder=0,
)
ax_rwtau.plot(
    data['t'], -np.ones_like(data['t']) * simulator.tau_max,
    ':', linewidth=4, color='C4', zorder=0,
)
ax_rwtau.grid()
ax_rwtau.legend(fontsize=16, ncol=4, loc='upper right')
ax_rwtau.tick_params(labelsize=14)


# Set x-axis properties (only need to do this on the last
# subplot since all subplots share the same x-axis)
ax_rwtau.set_xlabel('time (s)', fontsize=20)
ax_rwtau.set_xlim([data['t'][0], data['t'][-1]])
ax_rwtau.set_ylim(-1.2 * simulator.tau_max, 1.2 * simulator.tau_max)

# Make the arrangement of subplots look nice
fig.tight_layout()

Save the figure (without transparent background).

In [ ]:
fig.savefig('my_figure.png', facecolor='white', transparent=False)

Plot star positions in image. This is not necessarily a useful thing to plot. The point is to show how to access measurements of star position from logged data.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9, 9))
for i in range(len(simulator.stars)):
    y = data['star_meas'][:, 2 * i]
    z = data['star_meas'][:, 2 * i + 1]
    ax.plot(-y, z, label=f'star {i}', linewidth=0.5, marker='.', markersize=10)
ax.grid()
ax.legend(fontsize=16)
ax.tick_params(labelsize=14)
ax.set_xlim(-1., 1.)
ax.set_ylim(-1., 1.)